# Hangman game: PLAY AGAINST NLTK

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 


In [ ]:
play(4)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ length 11
You have 4 attempts remaining.

Enter your guess:


 a


Guess is a
Good guess: _ _ _ a _ _ _ _ _ _ _
You have 4 attempts remaining.

Enter your guess:


 c


Guess is c
Sorry, try again.
You have 3 attempts remaining.

Enter your guess:


...

Guess is v
Already guessed this before.
You have 1 attempts remaining.

Enter your guess:


 v


Guess is v
Already guessed this before.
Out of guesses. The word was metalsmiths

YOUR MISTAKES: 4

Do you want NLTK to play?:y/n


 y


Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ length 11
You have 4 attempts remaining.
Guess is e
Good guess: _ e _ _ _ _ _ _ _ _ _
You have 4 attempts remaining.
Guess is i
Good guess: _ e _ _ _ _ _ i _ _ _
You have 4 attempts remaining.
Guess is a
Good guess: _ e _ a _ _ _ i _ _ _
You have 4 attempts remaining.
Guess is s
Good guess: _ e _ a _ s _ i _ _ s
You have 4 attempts remaining.
Guess is n
Sorry, try again.
You have 3 attempts remaining.
Guess is r
Sorry, try again.
You have 2 attempts remaining.
Guess is t
Good guess: _ e t a _ s _ i t _ s
You have 2 attempts remaining.
Guess is o
Sorry, try again.
You have 1 attempts remaining.
Guess is l
Good guess: _ e t a l s _ i t _ s
You have 1 attempts remaining.
Guess is c
Sorry, try again.
Out of guesses. The word was metalsmiths

NLTK MISTAKES: 4

PRESS ANY KEY TO PLAY AGAIN OR N TO STOP:


In [2]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('\nEnter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

In [3]:
from nltk.corpus import brown
import numpy as np
import re
# get unique word types ocurring in the Brown corpus selecting only words with alpahbetic characters - lowercase
brown_words = set([word.lower() for word in brown.words()])
words=list(set([word.lower() for word in brown_words if re.match(r'^[a-z]+$',word)]))
# split for training and testing sets
np.random.shuffle(words)

In [4]:
import random
from IPython.display import clear_output

def play(att):
    attempts=att
    while True:
        word=random.choice(words)
        my_mistakes=hangman(word, human, attempts, True)
        print('\nYOUR MISTAKES:',my_mistakes)
        print('\nDo you want NLTK to play?:y/n')
        try:
            play_nltk= raw_input().lower().strip() # python 3
        except NameError:
            play_nltk= input().lower().strip() # python 2
        if play_nltk.lower()=='y'  :  
            nltk_mistakes=hangman(word, ngram_guesser, attempts, True,lambdas=[0.01]*10)
            print('\nNLTK MISTAKES:',nltk_mistakes)

        print('\nPress any key to play again or n to stop:'.upper())
        try:
            play= raw_input().lower().strip() # python 3
        except NameError:
            play= input().lower().strip() # python 2


        if (play.lower()=='n'):
            break
        else:
            clear_output()
    
    
        

In [5]:
from nltk.corpus import brown
import numpy as np
import re
# get unique word types ocurring in the Brown corpus selecting only words with alpahbetic characters - lowercase
brown_words = set([word.lower() for word in brown.words()])
words=list(set([word.lower() for word in brown_words if re.match(r'^[a-z]+$',word)]))
# split for training and testing sets
np.random.shuffle(words)
test_set=words[0:1000]
train_set=words[1000:len(words)]
print(len(test_set))
print(len(train_set))

1000
39234


In [6]:
# guessing method using the same input arguments as <human> method and returning a character
from string import ascii_lowercase

def trivial_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed] # from all letters in alphabet
    return np.random.choice(letter_available)
            
def evaluate_model(guesser,dataset,**guesser_args):
    verbose=False
    max_mistakes=26
    mistakes=0
    for word_test in dataset:   
        mistakes+=hangman(word_test, guesser, max_mistakes, verbose,**guesser_args)
      
    avg_mistakes=mistakes/len(dataset)
    return avg_mistakes
    


print(evaluate_model(trivial_guesser,test_set))

In [7]:
# get the frequencies of characters over all training words
from collections import defaultdict
from collections import Counter

## Code based on from WSTA_N11_n-gram_language_models.ipynb


def get_unigram_counts(words):
    unigram_counts = Counter()
    
    # collect initial unigram statistics
    for word in words:
        for letter in word:
                unigram_counts[letter] += 1

                
    return unigram_counts

## end of copied code

unigram_counts=get_unigram_counts(train_set)



To implement the Unigram model we need to calculate the probabilites for every character $c_i$:<br/>

$P_{unigram}(c_i)=\frac{count(c_i)}{M}$<br/>
where $M$ is the total number of characters

In [8]:
def unigram_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    total_counts = float(sum(unigram_counts.values()))
    unigram_probs={}
    vocab_size = len(unigram_counts)
    # calculate the probability for all letter available
    for letter in letter_available:
        unigram_probs[letter]=(unigram_counts[letter]+1)/(total_counts+vocab_size) #laplace smoothing
    
    # get the max probability 
    letter_choice=max(unigram_probs, key=unigram_probs.get)
    return letter_choice

        
# print the average number of mistakes the unigram method makes over the test set.
print(evaluate_model(unigram_guesser,test_set))

10.497


To implement the Unigram model conditioned we need to calculate the probabilites for every character $c_i$:<br/>

$P_{unigram}(c_i|n)=\frac{count(c_i,n)}{M}$<br/>
where $M$ is the total number of characters and $n$ is a fixed lenght for a word

In [9]:
# implement conditioning unigram model on the lengh of the word - different models according the lenght
# be careful with unseen lenghts in training
# create guessing function using the new model and print performance
def get_conditional_unigram_counts(words):    
    unigram_lenght_counts=defaultdict(Counter)
    total_counts=0.0
    
    # collect initial unigram statistics
    for word in words:
        for letter in word:
            unigram_lenght_counts[len(word)][letter] += 1
            
                
    return unigram_lenght_counts

unigram_lenght_counts=get_conditional_unigram_counts(train_set)

def unigram_conditional_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    total_counts = float(sum(unigram_lenght_counts[len(mask)].values()))
    unigram_probs={}
    vocab_size = len(unigram_lenght_counts)
    # calculate the probability for all letter available
    for letter in letter_available:
        # if the word len is not in the training set total_counts=0. 
        if (total_counts!=0):
            unigram_probs[letter]=(unigram_lenght_counts[len(mask)][letter]+1)/(total_counts+vocab_size)
        else:
            # out-of-vocabulary words = 1 / |V|
            vocab_size = len(unigram_lenght_counts)
            zerogram_prob = (1 / float(vocab_size)) 
            unigram_probs[letter]=zerogram_prob 
            
            
    # get the max probability 
    letter_choice=max(unigram_probs, key=unigram_probs.get)
    return letter_choice

In [10]:
# print the average number of mistakes the unigram method makes oert the test set.
print(evaluate_model(unigram_conditional_guesser,test_set))

10.481


In [11]:
import math

def convert_word(word,n):
    start=[]
    end=[]
    start_index=1
    
    # padding with sentinent symbols
        
    while start_index<n:
        start.append("<s"+str(start_index)+">")
        start_index+=1
    
    end_index=n
    while end_index>1:
        end.append("</s"+str(end_index-1)+">")
        end_index-=1
    
    return start + [l.lower() for l in word] + end

def get_ngram_counts(words,n):
    ngram_counts = defaultdict(Counter)
    
    # collect bigram counts
    for word in words:
        word = convert_word(word,n)
        if (n<=len(word)):
            n_grams=[word[i:i+n] for i in range(len(word)-(n-1))]
            for n_gram in n_grams:
                if (n==2):
                    ngram_counts[n_gram[0]][n_gram[1]]+=1
                elif (n==3):
                    ngram_counts[(n_gram[0],n_gram[1])][n_gram[2]]+=1
                elif (n==4):
                    ngram_counts[(n_gram[0],n_gram[1],n_gram[2])][n_gram[3]]+=1
                elif (n==5):
                    ngram_counts[(n_gram[0],n_gram[1],n_gram[2],n_gram[3])][n_gram[4]]+=1

    return ngram_counts





In [12]:
# train ngram where n = 2...5.
bigram_counts=get_ngram_counts(train_set,2)
trigram_counts=get_ngram_counts(train_set,3)
fourgram_counts=get_ngram_counts(train_set,4)
fivegram_counts=get_ngram_counts(train_set,5)

# dictionary for query in the guesser
ngram_counts = {1 : unigram_counts,
           2 : bigram_counts,
           3 : trigram_counts,
           4 : fourgram_counts,
           5 : fivegram_counts
    }

In [13]:
def get_ngram_prob(letter,context,n,lambdas):
    
    
    prob=0
    
    # create tuples to make a query with context
    if (n>1):
        if (len(context)>1):
            conditional=tuple(context)
        elif(len(context)==1):
            conditional=context[0]
        else:
            conditional=tuple() # no context at all
    
            

    if (n>4):
        
        # calculate the fivegram
        fivegram_count=ngram_counts[5][conditional][letter]*lambdas[5]
        fivegram_total_count=float(sum(ngram_counts[5][conditional].values()))
        if fivegram_total_count!=0:
            interp_prob_fivegram=fivegram_count/fivegram_total_count
        else:
            interp_prob_fivegram=0
            lambdas[4]+=lambdas[5] #if count is 0 I will give the lambda weight to the next ngram
            
        prob+=interp_prob_fivegram
        
    if (n>3):
        # calculate the fourgram
        fourgram_count=ngram_counts[4][conditional][letter]*lambdas[4]
        fourgram_total_count=float(sum(ngram_counts[4][conditional].values()))
        if fourgram_total_count!=0:
            interp_prob_fourgram=fourgram_count/fourgram_total_count
        else:
            interp_prob_fourgram=0
            lambdas[3]+=lambdas[4] #if count is 0 I will give the lambda weight to the next ngram
            
        prob+=interp_prob_fourgram
    
    if (n>2):
        # calculate the trigram
        trigram_count=ngram_counts[3][conditional][letter]*lambdas[3]
        trigram_total_count=float(sum(ngram_counts[3][conditional].values()))
        if trigram_total_count!=0:
            interp_prob_trigram=trigram_count/trigram_total_count
        else:
            interp_prob_trigram=0
            lambdas[2]+=lambdas[3] #if count is 0 I will give the lambda weight to the next ngram
        
        prob+=interp_prob_trigram
        
    if (n>1):
        # calculate the bigram
        bigram_count=ngram_counts[2][conditional][letter]*lambdas[2]
        bigram_total_count=float(sum(ngram_counts[2][conditional].values()))
        if bigram_total_count!=0:
            interp_prob_bigram=bigram_count/bigram_total_count
        else:
            interp_prob_bigram=0
            lambdas[1]+=lambdas[2] #if count is 0 I will give the lambda weight to the next ngram
        
        prob+=interp_prob_bigram
    
    if (n>0):
        unigram_count=ngram_counts[1][letter]*lambdas[1]
        unigram_total_count=float(sum(ngram_counts[1].values()))
        if unigram_total_count!=0:
            interp_prob_unigram=(unigram_count)/(unigram_total_count) # not smoothed with laplace like the unigram model       
        else:
            
            # out-of-vocabulary words = 1 / |V|
            lambdas[0]+=lambdas[1]
            vocab_size = len(unigram_total_count)
            interp_prob_unigram = (1 / float(vocab_size)) * lambdas[0]
        prob+=interp_prob_unigram
        
        
    return math.log(prob)
     

In [14]:
def ngram_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    blank_probs={}
    letter_probs={}
    probs=[]
    if kwargs.get('n'):       
        n=kwargs.get('n')
    else:
        n=1
        
    if kwargs.get('lambdas'):       
        lambdas=kwargs.get('lambdas')
    else:
        lambdas={1:1}
    word=convert_word(mask,2)
    letter_probs=defaultdict(float)
    for i in range(len(word)):
        if (word[i]=="_"):
            context=[]
            j=i-1
            context_len=1
            while word[j]!="_" and j>=0 and context_len<n:
                context.insert(0,word[j])
                j-=1
                context_len+=1
            
            for letter in letter_available:
                probability=get_ngram_prob(letter,context.copy(),n,lambdas.copy())
                letter_probs[letter]+=probability
                
              
    letter_choice=max(letter_probs, key=letter_probs.get)
    return letter_choice
